In [1]:
import datasets
from functools import reduce
import spacy
from spacy import displacy
import wp


In [2]:
import warnings
#warnings.filterwarnings("ignore", category=FutureWarning, module="spacy_transformers.layers")
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.utils")
warnings.filterwarnings("ignore", category=FutureWarning, module="thinc.shims")
#warnings.filterwarnings("ignore", category=UserWarning, module="spacy.util")
#warnings.filterwarnings("ignore", category=UserWarning, module="spacy_transformers.layers")

In [4]:
wp_ds = wp_ds.map(wp.preprocess, num_proc=6)

Map (num_proc=6):   0%|          | 0/272600 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/15138 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/15620 [00:00<?, ? examples/s]

In [6]:
nlp = spacy.load("en_core_web_trf")
analysis = nlp.analyze_pipes(pretty=True)

/home/igor/.pyenv/versions/3.11.10/envs/gah-env/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/igor/.pyenv/versions/3.11.10/envs/gah-env/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/igor/.pyenv/versions/3.11.10/envs/gah-env/lib/python3.11/site-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   transformer       doc._.trf_data                                      False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

In [7]:
# Se precisar de um Tokenizer customizado...
#from spacy.attrs import ORTH, NORM
#from spacy.tokenizer import Tokenizer
#tokenizer = Tokenizer(nlp.vocab)
#case = [{ORTH: "do"}, {ORTH: "n't", NORM: "not"}]
#tokenizer.add_special_case("don't", case)

In [ ]:
text = wp['train']['story'][0]

doc = nlp(text)

In [12]:
sents = list(doc.sents)
chunks = list(doc.noun_chunks)

In [15]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}
displacy.render(sents[0], style="dep", options=options, jupyter=True)

In [16]:
fmt = "{:15} {:10} {:10} {:15} {}"
print(fmt.format("Text", "Root Text", "Root Dep", "Root Head Text", "Children of head"))
for chunk in chunks[:4]:
    print(fmt.format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text, [child for child in chunk.root.head.children] ))

Text            Root Text  Root Dep   Root Head Text  Children of head
I               I          nsubj      walked          [times, have, I, on, ..]
ruins           ruins      pobj       on              [ruins]
the remainings  remainings appos      ruins           [,, remainings, loved]
places          places     pobj       of              [places]


In [17]:
trfdata = doc._.trf_data

trfdata.all_outputs[0].data, trfdata.all_outputs[0].data.shape

(array([[ 0.8684267 , -0.29793227, -0.00327166, ..., -1.4163585 ,
         -1.0987628 ,  0.15949313],
        [ 0.49095225,  0.8099425 , -0.087525  , ..., -0.09679326,
          0.00365211,  1.0270348 ],
        [ 0.32219228,  0.3153325 , -0.8179532 , ...,  1.6104504 ,
         -1.3762125 ,  0.07949264],
        ...,
        [ 0.23040941,  0.9806066 , -0.2744871 , ..., -1.0708449 ,
         -0.09195751,  1.1454837 ],
        [-0.7872363 , -0.8392252 , -0.8610389 , ...,  0.06520373,
         -0.02104913, -0.26914716],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 (686, 768))

In [18]:
# DO not work in last version of en_core_web_trf
#print(trfdata.tokens['input_texts'][0][:10])
#print(trfdata.tokens['input_texts'][0][-10:])#
#print(trfdata.tokens['input_texts'][1][:10])

# # Show all entries that are like the given id
# given_id = 14
# token_locations = np.where(trfdata.wordpieces.input_ids[0] == given_id)[0]

# tokens_on_doc = np.array(trfdata.wordpieces.strings[0])[token_locations]
# tokens_on_doc

# np.all(np.array(trfdata.wordpieces.strings) == np.array(trfdata.tokens['input_texts']))

# np.all(trfdata.align.dataXd == trfdata.align.data[:,0])

# np.all(trfdata.model_output['last_hidden_state'] == trfdata.tensors[0])

# aligns = trfdata.align.dataXd
# lengths = trfdata.align.lengths
# for pos,length in zip(aligns,lengths):
#     print(doc[pos-1:pos+length-1], end=" ")

In [19]:
trfdata.to_dict()['all_outputs'][-1][0]

array([[ 0.8684267 , -0.29793227, -0.00327166, ..., -1.4163585 ,
        -1.0987628 ,  0.15949313],
       [ 0.49095225,  0.8099425 , -0.087525  , ..., -0.09679326,
         0.00365211,  1.0270348 ],
       [ 0.32219228,  0.3153325 , -0.8179532 , ...,  1.6104504 ,
        -1.3762125 ,  0.07949264],
       ...,
       [ 0.23040941,  0.9806066 , -0.2744871 , ..., -1.0708449 ,
        -0.09195751,  1.1454837 ],
       [-0.7872363 , -0.8392252 , -0.8610389 , ...,  0.06520373,
        -0.02104913, -0.26914716],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [20]:
# Extract chunks from the ith-sentence
ith = 1
sentence = sents[ith]
chunksents = []
for chunk in chunks:
    if chunk.start < sentence.start:
        continue
    if chunk.end > sentence.end:
        break
    else:
        print(sentence.start, chunk.start, chunk.end, sentence.end)
        chunksents.append(chunk)

print(sentence)
print(chunksents)

21 23 24 65
21 29 30 65
21 32 34 65
21 35 38 65
21 42 45 65
21 45 46 65
21 48 49 65
21 50 51 65
21 53 54 65
21 55 57 65
21 57 58 65
21 62 64 65
At first I was scared, each time I could feel my city, my current generation collapse, break into the black hole that thrives within it, I could feel humanity, the way I'm able to feel my body..
[I, I, my city, my current generation, the black hole, that, it, I, humanity, the way, I, my body]


In [22]:
for token in doc:
    if (token.i+1) < len(doc) and doc[token.i + 1].is_sent_start:
        print(token)

..
..
.


.
,
.



.
.
.
.
..
.
.
.
.
.



''
.



.
.
?
.
